# 1. Prediction Task
# Introduction

In [1]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
import os
import pandas as pd
import numpy as np  
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import ttest_rel
from scipy.stats import randint

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from math import sqrt




In [2]:
# Load data
dataset1 = pd.read_csv('DataTrain-2.csv')
print(dataset1.head())

FileNotFoundError: [Errno 2] No such file or directory: 'DataTrain-2.csv'

In [ ]:
dataset1.dtypes

In [ ]:
print(dataset1.columns)

In [ ]:
print(dataset1.describe())

# 1.1 Modeling
## 1.1.1

In [ ]:
max_cycles = dataset1.groupby('engine_id')['cycle'].max().reset_index()
max_cycles.rename(columns={'cycle': 'max_cycle'}, inplace=True)
dataset1 = dataset1.merge(max_cycles, on='engine_id', how='left')
dataset1['RUL'] = dataset1['max_cycle'] - dataset1['cycle']

In [ ]:
dataset1['RUL'] ,max_cycles, dataset1

In [ ]:
max_cycles = dataset1.groupby('engine_id')['cycle'].max().reset_index()
max_cycles.rename(columns={'cycle': 'max_cycle'}, inplace=True)

plt.figure(figsize=(10, 6))
plt.hist(max_cycles['max_cycle'], bins=50, alpha=0.75, color='blue', edgecolor='black')
plt.title('Maximum Remaining Useful Life (Max RUL) per Engine')
plt.xlabel('Maximum Remaining Useful Life (Max RUL)')
plt.ylabel('Count of Engines')
plt.grid(True)
plt.show()

In [ ]:
# import matplotlib.pyplot as plt

#histogram for each sensor val
dataset1[[f'sensor_val{i+1}' for i in range(21)]].hist(bins=50, figsize=(15, 20))
plt.suptitle('Distribution of Sensor Values')
plt.show()

In [ ]:
# import seaborn as sns

# Correlation matrix
correlation_matrix = dataset1.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Features and RUL')
plt.show()


This correlation matrix helps us find which features are most linked to RUL, making them useful for predicting outcomes in models. It also helps spot multicollinearity, where high correlations between features can cause problems like overfitting or giving too much weight to certain features in the model.

In [ ]:
# Analyse the correlation of RUL with other features
correlation_rul = correlation_matrix['RUL'].sort_values(ascending=False)
correlation_rul

In [ ]:
#scatter plot
fig, axs = plt.subplots(7, 3, figsize=(15, 20))
axs = axs.flatten()

# Plot each sensor against cycle and RUL
for i in range(21):
    ax = axs[i]
    sensor_col = f'sensor_val{i+1}'  # Column name for each sensor
    ax.scatter(dataset1['cycle'], dataset1[sensor_col], alpha=0.5, label='Cycle')
    ax.scatter(dataset1['RUL'], dataset1[sensor_col], alpha=0.5, label='RUL')
    ax.set_xlabel('Cycle / Remaining Useful Life')
    ax.set_ylabel(f'Sensor Value {i+1}')
    ax.set_title(f'Sensor Value {i+1} across Cycles and RUL')
    ax.legend()

plt.tight_layout()
plt.show()


In this visualisation, the scatter plot is used to visually explore how Sensor Values behaves concerning 'cycle' and 'RUL' in the dataset. It is essential for quickly understanding relationships between variables like sensor readings and time-related metrics such as 'Cycle' or 'RUL'. It help identify patterns, outliers, and correlations visually, aiding in feature selection for predictive models.

In [ ]:
# Checking for missing values in the dataset
dataset1.isnull().sum()

In [ ]:
# generate a feature  for the remaining cycles for each cycle based on the max cycle number for each engine id in dataset
dataset1['RUL'] = dataset1['max_cycle'] - dataset1['cycle']
dataset1.shape

In [ ]:
# Dropping colun not useful fro training
features = dataset1.drop(columns=['RUL', 'engine_id', 'max_cycle' ])
target = dataset1['RUL']

In [ ]:
target.shape, features.shape

### Test and Training Split

In [ ]:
# from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
X_test.shape, X_train.shape, y_test.shape, y_train.shape

### RFM Training

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import mean_squared_error

# base model
model = RandomForestRegressor(n_estimators=100, max_depth=20, min_samples_split=5, min_samples_leaf=2,random_state=42)
model.fit(X_train, y_train)
# Predict RUL on the test data
y_pred = model.predict(X_test)
y_pred

#Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", rmse)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

# Calculate the root mean squared error
rmse_scorer = make_scorer(lambda y_true, y_pred: sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

# Perform cross-validation
scores = cross_val_score(model, X_train, y_train, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring=rmse_scorer)



In [ ]:
# Output 
print("Cross-validation scores:", scores)
print("Mean RMSE:", scores.mean())
print("Standard deviation:", scores.std())


In [ ]:
print(model.get_params())

In [ ]:
# Get the feature importances
importances = model.feature_importances_
# CreateDataFrame to store feature importances
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
# Sort DataFrame by importance: descending order
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

print(feature_importance_df)

In [ ]:
### Filter 0 importance feature for next models

features_to_keep = feature_importance_df[feature_importance_df['Importance'] > 0]['Feature']
X_train = X_train[features_to_keep]
X_test = X_test[features_to_keep]

print(feature_importance_df)

### Gridsearch

In [ ]:
# Define grid search
param_grid = {
    'n_estimators': [70, 80, 90, 100],
    'ccp_alpha': [0],
    'max_leaf_nodes': [None],
    'max_samples': [None],
    'max_features': [1],
    'max_depth': [None],
    'random_state': [18],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True]
}

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters from Grid Search:", grid_search.best_params_)

In [ ]:
# Evaluate model with best parameters from grid search
model = grid_search.best_estimator_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)

print("Grid Search - MAE:", mae, "RMSE:", rmse, "MSE:", mse)

### Random Search

In [ ]:
# Define randomized search
param_dist = {
    'n_estimators': randint(70, 100),
    'ccp_alpha': [0],
    'max_leaf_nodes': [None],
    'max_samples': [None],
    'max_features': [1],
    'max_depth': [None],
    'random_state': [18],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4),
    'bootstrap': [True]
}

# Perform randomized search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=5, n_jobs=-1)
random_search.fit(X_train, y_train)
print("Best parameters from Randomized Search:", random_search.best_params_)

In [ ]:
# Evaluate model with best parameters from randomized search
model = random_search.best_estimator_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)

print("Randomized Search - MAE:", mae, "RMSE:", rmse, "MSE:", mse)

In [ ]:
# Setup RMSE scorer for cross-validation
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Cross-validation using the final model
scores = cross_val_score(model, X_train, y_train, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring=make_scorer(rmse_scorer, greater_is_better=False))
print("Cross-validation scores:", scores)
print("Mean RMSE:", scores.mean())
print("Standard deviation:", scores.std())

In [ ]:
best_model_from_grid_search = grid_search.best_estimator_

## 1.2 Prediction

In [ ]:
# Load data
dataset2 = pd.read_csv('DataSchedule-2.csv')
dataset2

In [ ]:
# Assuming `original_features` is a list of feature names used during model training
original_features = X_train.columns.tolist()

# Check for any missing features and fill with zero or appropriate values
for feature in original_features:
    if feature not in dataset2.columns:
        dataset2[feature] = 0  # Or use another placeholder value like the mean of the column

# Remove any extra features not used during training
drop_eid = dataset2[original_features]

# Predict using the best model from grid search
pred_rul = best_model_from_grid_search.predict(drop_eid)
pred_rul = pred_rul.astype(int)

# Adding the predictions back to the original dataset
dataset2['RUL'] = pred_rul

# Optionally, if you need to keep the 'engine_id' for reference
dataset2['engine_id'] = dataset2['engine_id']

print(dataset2[['engine_id', 'RUL']])

In [ ]:
dataset3 = pd.read_csv('RUL_consultancy_predictions_A3-2.csv')
dataset3
# delimeter ;
dataset3 = pd.read_csv('RUL_consultancy_predictions_A3-2.csv', delimiter=';')
dataset3
# Convert 'RUL' and 'id' columns to appropriate data types (integers)
dataset3['RUL'] = dataset3['RUL'].astype(int)
dataset3['id'] = dataset3['id'].astype(int)
dataset3



In [ ]:
# Weighted Average to calculate single RUL
# Function to compute weighted average for each engine_id
def compute_weighted_avg(group):
    return group['weighted_rul'].sum() / group['weight'].sum()

weighted_rul_df = dataset2.copy()
weighted_rul_df['weight'] = weighted_rul_df['cycle'] / weighted_rul_df.groupby('engine_id')['cycle'].transform('max')
weighted_rul_df['weighted_rul'] = weighted_rul_df['RUL'] * weighted_rul_df['weight']
weighted_avg_rul = weighted_rul_df.groupby('engine_id').apply(compute_weighted_avg).reset_index()
weighted_avg_rul.columns = ['id', 'RUL']
weighted_avg_rul['RUL'] = weighted_avg_rul['RUL'].round().astype(int)
merged_results = pd.merge(weighted_avg_rul, dataset3, on='id', how='inner', suffixes=('_predicted', '_consultancy'))

predictions_df = merged_results.drop(columns=['RUL_consultancy'])
print(merged_results.head())

In [ ]:
# Extract the RUL columns
rul_predicted = merged_results['RUL_predicted']
rul_consultancy = merged_results['RUL_consultancy']

# Calculate metrics
mae = mean_absolute_error(rul_consultancy, rul_predicted)
rmse = mean_squared_error(rul_consultancy, rul_predicted, squared=False)
mse = mean_squared_error(rul_consultancy, rul_predicted)

# Output
print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("Mean Squared Error:", mse)


In [ ]:
# Perform statistical tests paired with t-test
t_stat, p_value = ttest_rel(rul_consultancy, rul_predicted)

# Output
print("T-statistic:", t_stat)
print("P-value:", p_value)


In [ ]:
# Save prediction to a CSV file
predictions_df.to_csv('rul_predictions.csv', index=False)

# 2. Optimization Task

## 2.1 Genetic Algorithm 

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import time

In [ ]:
# read lr_predictions.csv and copy it to a new DataFrame
predictions = pd.read_csv('lr_predictions.csv')
predictions_copy = predictions.copy()
predictions_copy


In [ ]:
# In this task, the goal is to develop a maintenance schedule for the engines in data set 2. The output of Task 1.2 should be a list that contains a predicted RUL for each engine in data set 2. These predicted values will now be used as input in order to allocate teams of workers to perform maintenance.

In [ ]:
# We want to allocate workers to different engines in order to perform maintenance. Workers perform maintenance in teams, and teams can only work on one engine at a time. Also, two teams cannot work on the same engine simultaneously. There are different types of teams: teams of type A and teams of type B. There are G teams in total. In our case, assume that there are G = 4 teams: 2 teams of type A and 2 teams of type B. Assume that teams T1,T3 are type A, and teams T2,T4 are type B. The engines that need maintenance are numbered with indices from the set M= {1,...,M}, with M being the total number of engines. Teams of type A need μAj days to perform maintenance on engine j ∈ M, and teams of type B need μBj days. Teams of type A are more efficient, as μAj < μBj ∀j ∈M. In our case, M = 100. The values for μAj ,j ∈ M are as follows: μAj = 4,j ∈ {1,2,...,20}, (ii) μAj = 3,j ∈ {21,22,...,55}, (iii) μAj = 2,j ∈ {56,57,...,80}, and (iv) μAj = 8,j ∈ {81,82,...,100}. The values for μBj ,j ∈ M are as follows: μBj = μAj + 1,j ∈ {1,2,...,25}, (ii) μBj = μAj + 2,j ∈ {26,27,...,70}, and (iii) μBj = μAj + 1,j ∈{71,72,...,100}.

In [ ]:
# Define the total number of engines
M = 100
# Define the number of teams
G = 4
# Define the time horizon
T = 30
# Define the maximum daily penalty cost
MAX_DAILY_COST = 250



# Initialize lists to store μAj and μBj values for each engine
mu_A = [0] * M # mu_A is maintenance time for team A
mu_B = [0] * M # mu_B is maintenance time for team B
engine_costs = []

# Populate engine_costs with values according to the given rules
for j in range(1, M + 1):
    if j <= 20:
        engine_costs.append(4)
    elif 21 <= j < 31:
        engine_costs.append(3)
    elif 31 <= j < 46:
        engine_costs.append(2)
    elif 46 <= j < 81:
        engine_costs.append(5)
    else:
        engine_costs.append(6)

# Populate mu_A with values according to the given rules
for j in range(1, M + 1):
    if 1 <= j <= 20:
        mu_A[j-1] = 4
    elif 21 <= j <= 55:
        mu_A[j-1] = 3
    elif 56 <= j <= 80:
        mu_A[j-1] = 2
    elif 81 <= j <= 100:
        mu_A[j-1] = 8

# Populate mu_B based on mu_A values and the given rules
for j in range(1, M + 1):
    if 1 <= j <= 25:
        mu_B[j-1] = mu_A[j-1] + 1
    elif 26 <= j <= 70:
        mu_B[j-1] = mu_A[j-1] + 2
    elif 71 <= j <= 100:
        mu_B[j-1] = mu_A[j-1] + 1

# Print results
print("Engine Costs:", engine_costs)
print("μA values:", mu_A)
print("μB values:", mu_B)


In [ ]:
# Define the number of teams of type A and type B
G_A = 2
G_B = 2

# Define the teams of type A and type B
teams_A = ['T1', 'T3']
teams_B = ['T2', 'T4']

# Define the total number of teams
teams = teams_A + teams_B

# Output the teams
print("Teams of type A:", teams_A)
print("Teams of type B:", teams_B)
print("All teams:", teams)

In [ ]:
# 2.1 Genetic Algorithm - First, you are going to develop a Genetic Algorithm to solve the optimization problem. Company X is going to allocate teams to different engines in order to perform maintenance. Assume that we are currently at day t = 1 and that company X wants to allocate teams to engines in order to minimize penalty costs for a planning horizon of T = 30. In other words, company X wants to allocate teams to engines that have a predicted safety due date of less than 30.
# 2.1.1 Write a Genetic Algorithm that solves the problem for company X

In [ ]:
# Create the creator for the fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Function to create a feasible individual
def create_individual():
    individual = []
    for engine_id in range(1, len(predictions_copy) + 1):
        if predictions_copy.loc[predictions_copy['engine_id'] == engine_id, 'RUL'].values[0] < T:
            team_type = random.choice(['A', 'B'])
            start_day = random.randint(1, T)
            individual.append((engine_id, team_type, start_day))
    return individual

# Function to convert an individual to a schedule and return DataFrame
def individual_to_schedule(individual):
    penalty_cost = 0
    schedule = []
    total_penalty_cost = 0
    for engine_id, team_type, start_day in individual:
        predicted_rul = predictions_copy.loc[predictions_copy['engine_id'] == engine_id, 'RUL'].values[0]
        if team_type == 'A':
            maintenance_time = mu_A[engine_id - 1]
        else:
            maintenance_time = mu_B[engine_id - 1]
        end_day = start_day + maintenance_time - 1
        if end_day > predicted_rul:
            penalty_cost = (end_day - predicted_rul) * MAX_DAILY_COST
            total_penalty_cost += penalty_cost
        schedule.append({'RUL': predicted_rul, 'Engine_id': engine_id, 'Team': team_type, 'Start_date': start_day, 'End_date': end_day, 'Penalty_cost': penalty_cost, 'Total_penalty_cost': total_penalty_cost})
    return pd.DataFrame(schedule), total_penalty_cost
    
# Function to calculate penalty cost per engine based on maintenance schedule
def calculate_penalty_cost(engine_id, team_type, start_day):
    predicted_rul = predictions_copy.loc[predictions_copy['engine_id'] == engine_id, 'RUL'].values[0]
    if team_type == 'A':
        maintenance_time = mu_A[engine_id - 1]
    else:
        maintenance_time = mu_B[engine_id - 1]
    end_day = start_day + maintenance_time - 1
    penalty = 0

    if end_day > predicted_rul:
        overdue_days = end_day - predicted_rul
        penalty = min(MAX_DAILY_COST, engine_costs[engine_id - 1] * overdue_days ** 2)

    return penalty

# Fitness function: total penalty cost for the individuaL
def calculate_fitness(individual):
    total_penalty_cost = 0
    for engine_id, team_type, start_day in individual:
        penalty_cost = calculate_penalty_cost(engine_id, team_type, start_day)
        total_penalty_cost += penalty_cost
    return (total_penalty_cost,)

# Feasibility function: check if the individual satisfies all constraints
def is_feasible(individual):
    engine_ids = [engine_id for engine_id, _, _ in individual]
    return len(engine_ids) == len(set(engine_ids))

def cxTwoPoint(ind1, ind2):
    size = min(len(ind1), len(ind2))
    print(f"Individual lengths: {len(ind1)}, {len(ind2)}")
    print(f"Size: {size}")
    
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    print(f"Crossover points: {cxpoint1}, {cxpoint2}")
    
    if cxpoint2 >= cxpoint1:
        ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    return ind1, ind2

In [ ]:

# Create the toolbox
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", calculate_fitness)
toolbox.decorate("evaluate", tools.DeltaPenalty(is_feasible, 10000))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=2)


# Create intial population
population = toolbox.population(n=50)
# Define statistics and hall of fame
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)
hof = tools.HallOfFame(1)

# Number of generations
num_generations = 250

# Run the genetic algorithm§
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.1, ngen=num_generations, stats=stats, halloffame=hof)

# Print best individual and its fitness values
best_individual = hof[0]
best_fitness = best_individual.fitness.values
print("Best individual is:", best_individual)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# historical data
gen = logbook.select("gen")
max_val = logbook.select("max")
avg_val = logbook.select("avg")
min_val = logbook.select("min")

# Calculate the coefficients of the linear regression
slope, intercept = np.polyfit(gen, avg_val, 1)
# Generate y-values based on the slope and intercept
trendline = np.polyval([slope, intercept], gen)

# Plot the historical data
plt.figure(figsize=(12, 6))
plt.plot(gen, max_val, label='Max Fitness')
plt.plot(gen, avg_val, label='Average Fitness')
plt.plot(gen, min_val, label='Min Fitness')

# Add a linear regression line for the average fitness
plt.plot(gen, trendline, color='r', linestyle='-', label='Average Fitness Trend')

plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Genetic Algorithm Performance')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### 2.2 Optimisation

In [ ]:
def run_ga(t, toolbox, T, num_generations):
    start_time = time.time()
    elapsed_time = 0
    best_individuals = []
    best_fitness_values = []

    while elapsed_time < t:  # 5 minutes
        pop = [toolbox.individual() for _ in range(1000)]  # Initialize population directly
        hof = tools.ParetoFront()
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean, axis=0)
        stats.register("std", np.std, axis=0)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)
        
        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.1,
                                   ngen=num_generations, stats=stats, halloffame=hof)
        
        if hof:
            best_individuals.append(hof[0])
            best_fitness_values.append(hof[0].fitness.values[0])

        elapsed_time = time.time() - start_time
    
    # Select the best of the best individuals
    best_index = np.argmin(best_fitness_values)
    best_individual = best_individuals[best_index]
    best_fitness = best_fitness_values[best_index]

    return best_individual, best_fitness

In [ ]:
t_in_seconds = 300 # 5 minutes
best_individual, best_fitness = run_ga(t_in_seconds, toolbox, T, num_generations)

print(f"\nBest individual: {best_individual}\nFitness: {best_fitness}")

In [ ]:
best_individual = hof[0] 
schedule_df, total_penalty_cost = individual_to_schedule(best_individual)

# Display the schedule DataFrame
print("Schedule:")
print(schedule_df)

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from deap import tools, algorithms

def run_ga_multiple_times(t, toolbox, num_generations):
    start_time = time.time()
    elapsed_time = 0
    best_fitness_values = []

    while elapsed_time < t:  # 5 minutes
        pop = [toolbox.individual() for _ in range(1000)]  # Initialize population directly
        hof = tools.ParetoFront()
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean, axis=0)
        stats.register("std", np.std, axis=0)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)
        
        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.1,
                                   ngen=num_generations, stats=stats, halloffame=hof)
        
        if hof:
            current_best_fitness = hof[0].fitness.values[0]
            best_fitness_values.append(current_best_fitness)

        elapsed_time = time.time() - start_time

    # Calculate the average best fitness
    average_best_fitness = np.mean(best_fitness_values)

    return average_best_fitness

# Call the function to run the genetic algorithm 30 times for at most 5 minutes
average_best_fitness = run_ga_multiple_times(300, toolbox, num_generations)

In [ ]:
# Display the schedule DataFrame and total penalty costs
print("Schedule:")
print(schedule_df)


In [ ]:
### 2.3 Comparison 
# Run the Genetic Algorithm on the same problem, but with the RUL predictions of the consultancy company. Run it 30 times for at most 5 minutes, and plot the average-best-fitness.
# Create a new DataFrame with the consultancy RUL predictions
rul_df = pd.read_csv('RUL_consultancy_predictions_A3-2.csv', delimiter=';')
rul_df

In [ ]:
# Run the ga 30 times for 5 minutes and plot the average-fitness-values.
def run_ga_multiple_times_consultancy(t, toolbox, num_generations):
    start_time = time.time()
    elapsed_time = 0
    best_fitness_values = []

    while elapsed_time < t:  # 5 minutes
        pop = [toolbox.individual() for _ in range(1000)]  # Initialize population directly
        hof = tools.ParetoFront()
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean, axis=0)
        stats.register("std", np.std, axis=0)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)
        
        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.1,
                                   ngen=num_generations, stats=stats, halloffame=hof)
        
        if hof:
            current_best_fitness = hof[0].fitness.values[0]
            best_fitness_values.append(current_best_fitness)

        elapsed_time = time.time() - start_time

    # Calculate the average best fitness
    average_best_fitness = np.mean(best_fitness_values)

    return average_best_fitness

In [ ]:
#priunt the best individual
best_individual2, best_fitness = run_ga(t_in_seconds, toolbox, T, num_generations)
# Display the schedule DataFrame
schedule_df = individual_to_schedule(best_individual2)
print("Schedule:")
print(schedule_df)


In [ ]:
# plot the final average best fitness
average_best_fitness_consultancy = run_ga_multiple_times_consultancy(300, toolbox, num_generations)
print(f"Average Best Fitness: {average_best_fitness_consultancy}")


In [ ]:
# Plot the average best fitness values for the two cases
plt.figure(figsize=(12, 6))
plt.plot([average_best_fitness] * 30, label='Company X')
plt.plot([average_best_fitness_consultancy] * 30, label='Consultancy')
plt.xlabel('Run')
plt.ylabel('Average Best Fitness')
plt.title('Average Best Fitness Values for Company X and Consultancy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()